As currently written, will output the foldability for all $2 \times n$ assignments. The last bit of the code can be modified to run other experiments, like checking a specific assignment, or outputing the count of ways to fold an assignment or assignments, etc.

In [1]:
'''Code from other file'''

import itertools
import numpy as np

n = 7

def get_bottom_edges(n):
    return [(i, i+1) for i in range(0, 2 * n, 2)]

def get_left_edges(n):
    return [(i, i+2) for i in range(0, 2 * n - 2, 4)] + [(i, i+2) for i in range(1, 2 * n - 2, 4)]

def get_right_edges(n):
    return [(i, i+2) for i in range(2, 2 * n - 2, 4)] + [(i, i+2) for i in range(3, 2 * n - 2, 4)]

def get_positive_faces(n):
    return list(range(0, 2*n, 4)) + list(range(3, 2*n, 4))

def get_negative_faces(n):
    return list(range(2, 2*n, 4)) + list(range(1, 2*n, 4))


adj = {}
for face in range(2 * n):
    adj[face] = []
    if face - 2 >= 0:
        adj[face].append(face-2)
    if face + 2 < n:
        adj[face].append(face+2)
    if face % 2 == 0:
        adj[face].append(face+1)
    if face % 2 == 1:
        adj[face].append(face-1)

bottom_edges = get_bottom_edges(n)
left_edges = get_left_edges(n)
right_edges = get_right_edges(n)
positive_faces = get_positive_faces(n)
negative_faces = get_negative_faces(n)

# check for crossing violations
def check_crossings(edges, ordering):
    for e1, e2 in itertools.combinations(edges, 2):
        b1 = min(ordering[e1[0]], ordering[e1[1]])
        t1 = max(ordering[e1[0]], ordering[e1[1]])
        b2 = min(ordering[e2[0]], ordering[e2[1]])
        t2 = max(ordering[e2[0]], ordering[e2[1]])

        if b1 < b2 < t1 < t2 or b2 < b1 < t2 < t1:
            # print("CROSSING", e1, e2, b1, t1, b2, t2)
            return False
    return True
    
def check_all_crossings(assignment, ordering):
    if not check_crossings(bottom_edges, ordering) or not check_crossings(left_edges, ordering) or not check_crossings(right_edges, ordering):
        return False

    return True

def convertToMV(assign):
    edge_assign = {}
    edge_assign[(0,1)] = -1
    for i in range(len(assign)):
        top_left = 2*i
        base = edge_assign[(top_left, top_left+1)]
        if assign[i] == 1:
            edge_assign[(top_left, top_left+2)] = -1*base
            edge_assign[(top_left+1, top_left+3)] = -1*base
            edge_assign[(top_left+2, top_left+3)] = -1*base
        if assign[i] == 2:
            edge_assign[(top_left, top_left+2)] = base
            edge_assign[(top_left+1, top_left+3)] = -1*base
            edge_assign[(top_left+2, top_left+3)] = base
        if assign[i] == 3:
            edge_assign[(top_left, top_left+2)] = base
            edge_assign[(top_left+1, top_left+3)] = base
            edge_assign[(top_left+2, top_left+3)] = -1*base
        if assign[i] == 4:
            edge_assign[(top_left, top_left+2)] = -1*base
            edge_assign[(top_left+1, top_left+3)] = base
            edge_assign[(top_left+2, top_left+3)] = base
    return edge_assign

def get_poset(assignment):
    poset = []
    # Loops over edges and adds them (using parity of edge and of face location)
    for edge in assignment:
        if (edge[0]%4 == 0 or edge[0]%4 == 3):
            if (assignment[edge] == -1):
                poset.append([edge[0], edge[1]])
            else:
                poset.append([edge[1], edge[0]])
        else:
            if (assignment[edge] == 1):
                poset.append([edge[0], edge[1]])
            else:
                poset.append([edge[1], edge[0]])
    
    return Poset((list(range(2 * n)), poset))

## Iterate over all possible linear extensions of the poset and check edges

def count(assignment):
    count = 0
    P = get_poset(assignment)
    for i in P.linear_extension().parent():
        if(check_all_crossings(assignment, np.argsort(i))):
            # print(i)
            count+= 1
    return count

def foldable(assignment):
    P = get_poset(assignment)
    perms = list(P.linear_extension().parent())
    np.random.shuffle(perms)
    for i in perms:
        if(check_all_crossings(assignment, np.argsort(i))):
            # print(i)
            return True
    return False

with open("2x7_foldable.txt", "w") as f:
    for rep in list(itertools.product([1, 2, 3, 4], repeat=n-1)):
        f.write(str(rep) + " " + str(foldable(convertToMV(rep))) + "\n")
    f.close()

KeyboardInterrupt: 